# Examining RB Health in the Proceeding Season Based on Workload

## Get Data

In [1]:
import nfl_data_py as nfl

In [2]:
years = range(1999, 2024 + 1)
s_type = "ALL"
df_seasons_all = nfl.import_seasonal_data(years, s_type)

In [3]:
df_roster = nfl.import_seasonal_rosters(years)

In [4]:
df_roster.columns

Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number',
       'status', 'player_name', 'first_name', 'last_name', 'birth_date',
       'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id',
       'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id',
       'sleeper_id', 'years_exp', 'headshot_url', 'esb_id', 'gsis_it_id',
       'smart_id', 'entry_year', 'rookie_year', 'draft_club', 'ngs_position',
       'week', 'game_type', 'status_description_abbr', 'football_name',
       'draft_number', 'age'],
      dtype='object')

In [5]:
df_roster_clean = df_roster[["season", "player_id", "player_name", "age"]]

In [6]:
df_roster_clean.query("player_id == '00-0034844'")

,season,player_id,player_name,age
44701,2018,00-0034844,Saquon Barkley,21.0
47076,2019,00-0034844,Saquon Barkley,22.0
49813,2020,00-0034844,Saquon Barkley,23.0
52439,2021,00-0034844,Saquon Barkley,24.0
55076,2022,00-0034844,Saquon Barkley,25.0
57904,2023,00-0034844,Saquon Barkley,26.0
60810,2024,00-0034844,Saquon Barkley,27.0


## Create DFs

In [7]:
df_carries = df_seasons_all.groupby(['player_id', 'season'], as_index=False).agg({
    'carries': 'sum',
    'receptions': 'sum',
    'games': 'max'
})
df_carries["touches"] = df_carries["carries"] + df_carries["receptions"]

In [8]:
df_carries_over_400 = df_carries.query("carries >= 400")
df_carries_over_400.head()

,player_id,season,carries,receptions,games,touches
645,00-0004304,2004,410,24,18,434
927,00-0005883,1999,424,55,20,479
928,00-0005883,2000,425,56,17,481
993,00-0006305,2003,403,58,18,461
1279,00-0008241,2000,404,64,17,468


In [9]:
df_rush_all = df_seasons_all.query("season_type == 'REG'")[["player_id", "season", "season_type", "carries", 
                                                            "rushing_yards", "rushing_tds", "rushing_first_downs", 
                                                            "rushing_epa", "games", "fantasy_points_ppr",
                                                            "receptions", "receiving_yards", "receiving_tds"
                                                           ]]

In [10]:
df_merged_one = df_rush_all.merge(
    df_roster_clean[['player_id', 'season', 'player_name', 'age']],
    on=['player_id', 'season'],
    how='left'
)

## Over 400 Carries

In [11]:
rb_dict = {}
for index, row in df_carries_over_400.iterrows():
    season = row["season"]
    player_id = row["player_id"]

    df_one = df_merged_one.query(f"player_id == '{player_id}' and season == {season}")
    df_two = df_merged_one.query(f"player_id == '{player_id}' and season == {season + 1}")
    
    player = df_one["player_name"].iloc[0]
    age = df_one["age"].iloc[0]
    rb_dict[(player, season)] = {}

    if len(df_one) == 1 and len(df_two) == 1:
        games_0 = df_one['games'].iloc[0]
        games_1 = df_two['games'].iloc[0]

        games_diff = games_1 - games_0
        rb_dict[(player, season)]["games_diff"] = games_diff

        yards_0 = df_one['rushing_yards'].iloc[0]
        yards_1 = df_two['rushing_yards'].iloc[0]

        yards_diff = yards_1 - yards_0
        rb_dict[(player, season)]["yards_diff"] = yards_diff

        ff_0 = df_one["fantasy_points_ppr"].iloc[0]
        ff_1 = df_two["fantasy_points_ppr"].iloc[0]

        ff_diff = ff_1 - ff_0
        rb_dict[(player, season)]["fantasy_points_ppr"] = ff_diff

        rb_dict[(player, season)]["seasons"] = (season, season + 1)
        rb_dict[(player, season)]["age"] = (age, age + 1)

In [12]:
rb_dict

{('Corey Dillon', 2004): {'games_diff': -5,
  'yards_diff': -902.0,
  'fantasy_points_ppr': -71.4,
  'seasons': (2004, 2005),
  'age': (29.0, 30.0)},
 ('Eddie George', 1999): {'games_diff': -3,
  'yards_diff': 178.0,
  'fantasy_points_ppr': 37.299999999999955,
  'seasons': (1999, 2000),
  'age': (25.0, 26.0)},
 ('Eddie George', 2000): {'games_diff': -1,
  'yards_diff': -516.0,
  'fantasy_points_ppr': -149.59999999999997,
  'seasons': (2000, 2001),
  'age': (26.0, 27.0)},
 ('Ahman Green', 2003): {'games_diff': -2,
  'yards_diff': -720.0,
  'fantasy_points_ppr': -156.4,
  'seasons': (2003, 2004),
  'age': (26.0, 27.0)},
 ('Edgerrin James', 2000): {'games_diff': -11,
  'yards_diff': -1028.0,
  'fantasy_points_ppr': -262.6,
  'seasons': (2000, 2001),
  'age': (22.0, 23.0)},
 ('Curtis Martin', 2004): {'games_diff': -6,
  'yards_diff': -962.0,
  'fantasy_points_ppr': -181.89999999999998,
  'seasons': (2004, 2005),
  'age': (31.0, 32.0)},
 ('Shaun Alexander', 2005): {'games_diff': -7,
  'yard

In [24]:
for (player, season) in rb_dict:
    if rb_dict[(player, season)]:
        print(player, season, rb_dict[(player, season)]["age"])
        print(rb_dict[(player, season)]["fantasy_points_ppr"])
        print()

Corey Dillon 2004 (29.0, 30.0)
-71.4

Eddie George 1999 (25.0, 26.0)
37.299999999999955

Eddie George 2000 (26.0, 27.0)
-149.59999999999997

Ahman Green 2003 (26.0, 27.0)
-156.4

Edgerrin James 2000 (22.0, 23.0)
-262.6

Curtis Martin 2004 (31.0, 32.0)
-181.89999999999998

Shaun Alexander 2005 (28.0, 29.0)
-234.4

Jamal Lewis 2003 (24.0, 25.0)
-158.60000000000002

Larry Johnson 2006 (26.0, 27.0)
-242.40000000000003

Arian Foster 2012 (26.0, 27.0)
-193.60000000000002

DeMarco Murray 2014 (26.0, 27.0)
-166.70000000000002



## Over 350 Carries

In [15]:
df_carries_over_350 = df_carries.query("carries >= 350")
df_carries_over_350.head()

,player_id,season,carries,receptions,games,touches
94,00-0000745,2005,370,57,17,427
95,00-0000745,2006,353,60,17,413
165,00-0001215,2000,352,13,16,365
573,00-0004052,2001,356,28,16,384
575,00-0004052,2003,382,16,18,398


In [16]:
rb_dict_350 = {}
for index, row in df_carries_over_350.iterrows():
    season = row["season"]
    player_id = row["player_id"]

    df_one = df_merged_one.query(f"player_id == '{player_id}' and season == {season}")
    df_two = df_merged_one.query(f"player_id == '{player_id}' and season == {season + 1}")
    
    player = df_one["player_name"].iloc[0]
    age = df_one["age"].iloc[0]
    rb_dict_350[(player, season)] = {}

    if len(df_one) == 1 and len(df_two) == 1:
        games_0 = df_one['games'].iloc[0]
        games_1 = df_two['games'].iloc[0]

        games_diff = games_1 - games_0
        rb_dict_350[(player, season)]["games_diff"] = games_diff

        yards_0 = df_one['rushing_yards'].iloc[0]
        yards_1 = df_two['rushing_yards'].iloc[0]

        yards_diff = yards_1 - yards_0
        rb_dict_350[(player, season)]["yards_diff"] = yards_diff

        ff_0 = df_one["fantasy_points_ppr"].iloc[0]
        ff_1 = df_two["fantasy_points_ppr"].iloc[0]

        ff_diff = ff_1 - ff_0
        rb_dict_350[(player, season)]["fantasy_points_ppr"] = ff_diff

        rb_dict_350[(player, season)]["seasons"] = (season, season + 1)
        rb_dict_350[(player, season)]["age"] = (age, age + 1)

In [17]:
rb_dict_350

{('Tiki Barber', 2005): {'games_diff': 0,
  'yards_diff': -198.0,
  'fantasy_points_ppr': -60.30000000000001,
  'seasons': (2005, 2006),
  'age': (30.0, 31.0)},
 ('Tiki Barber', 2006): {},
 ('Jerome Bettis', 2000): {'games_diff': -4,
  'yards_diff': -230.0,
  'fantasy_points_ppr': -49.620000000000005,
  'seasons': (2000, 2001),
  'age': (28.0, 29.0)},
 ('Stephen Davis', 2001): {'games_diff': -4,
  'yards_diff': -612.0,
  'fantasy_points_ppr': -58.50000000000003,
  'seasons': (2001, 2002),
  'age': (27.0, 28.0)},
 ('Stephen Davis', 2003): {'games_diff': -16,
  'yards_diff': -1352.0,
  'fantasy_points_ppr': -201.9,
  'seasons': (2003, 2004),
  'age': (29.0, 30.0)},
 ('Corey Dillon', 2004): {'games_diff': -5,
  'yards_diff': -902.0,
  'fantasy_points_ppr': -71.4,
  'seasons': (2004, 2005),
  'age': (29.0, 30.0)},
 ('Eddie George', 1999): {'games_diff': -3,
  'yards_diff': 178.0,
  'fantasy_points_ppr': 37.299999999999955,
  'seasons': (1999, 2000),
  'age': (25.0, 26.0)},
 ('Eddie George'

In [21]:
for (player, season) in rb_dict_350.keys():
    if rb_dict_350[(player, season)]:
        yards_diff = rb_dict_350[(player, season)]["yards_diff"]
        if yards_diff >= 0:
            print(player, season, rb_dict_350[(player, season)]["age"])
            print(yards_diff)
            print(rb_dict_350[(player, season)]["seasons"])
            print("-" * 50)

Eddie George 1999 (25.0, 26.0)
178.0
(1999, 2000)
--------------------------------------------------
Edgerrin James 1999 (21.0, 22.0)
216.0
(1999, 2000)
--------------------------------------------------
Edgerrin James 2003 (25.0, 26.0)
291.0
(2003, 2004)
--------------------------------------------------
Shaun Alexander 2004 (27.0, 28.0)
192.0
(2004, 2005)
--------------------------------------------------
Rudi Johnson 2004 (24.0, 25.0)
1.0
(2004, 2005)
--------------------------------------------------
LaDainian Tomlinson 2004 (25.0, 26.0)
129.0
(2004, 2005)
--------------------------------------------------
Marshawn Lynch 2013 (27.0, 28.0)
49.0
(2013, 2014)
--------------------------------------------------
Derrick Henry 2019 (25.0, 26.0)
487.0
(2019, 2020)
--------------------------------------------------


In [23]:
for (player, season) in rb_dict_350.keys():
    if rb_dict_350[(player, season)]:
        ff_diff = rb_dict_350[(player, season)]["fantasy_points_ppr"]
        if ff_diff >= 0:
            print(player, season, rb_dict_350[(player, season)]["age"])
            print(ff_diff)
            print(rb_dict_350[(player, season)]["seasons"])
            print("-" * 50)

Eddie George 1999 (25.0, 26.0)
37.299999999999955
(1999, 2000)
--------------------------------------------------
Edgerrin James 1999 (21.0, 22.0)
27.80000000000001
(1999, 2000)
--------------------------------------------------
Edgerrin James 2003 (25.0, 26.0)
40.200000000000045
(2003, 2004)
--------------------------------------------------
Edgerrin James 2004 (26.0, 27.0)
4.199999999999989
(2004, 2005)
--------------------------------------------------
Curtis Martin 1999 (26.0, 27.0)
56.32000000000002
(1999, 2000)
--------------------------------------------------
Shaun Alexander 2004 (27.0, 28.0)
54.0
(2004, 2005)
--------------------------------------------------
Rudi Johnson 2004 (24.0, 25.0)
16.69999999999999
(2004, 2005)
--------------------------------------------------
LaDainian Tomlinson 2002 (23.0, 24.0)
59.64000000000004
(2002, 2003)
--------------------------------------------------
LaDainian Tomlinson 2004 (25.0, 26.0)
32.15999999999997
(2004, 2005)
---------------------

In [20]:
print(len(rb_dict_350.keys()))

53


In [ ]:
df_carries_over_350.query("player_id == '00-0025399'")

In [ ]:
df_merged_one.query("player_name == 'Marshawn Lynch'")